In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import Input,Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionV3,VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,TopKCategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,TensorBoard

In [3]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
print ('Number of devices: %d' % strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


In [4]:
# Veri seti klasörünün yolu (Data klasörünüzün yolu)
data_dir = Path('Dataset/Train')
filepaths = list(data_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
train_df, test_df = train_test_split(images, train_size=0.75, shuffle=True, random_state=42)

In [5]:
batch_size = 4
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
        rescale=1./255,
        horizontal_flip=True,
        validation_split=0.2,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
) # %70 eğitim datası içinden ön eğitim kontrölü içinde %20 yani 140 adet ön test (daha önceden gördüğü veri üzerinden test yapmak için) veri ayırıyoruz

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    labels='inferred',
    label_mode='int',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=123,
    cros_to_aspect_ratio =False,
    follow_links=False,
    interpolation='bilinear',
    subset='training'
)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    label_mode='int',
    labels='inferred',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    interpolation='bilinear',
    shuffle=True,
    seed=123,
    follow_links=False,
    cros_to_aspect_ratio =False,
    subset='validation'
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    label_mode='int',
    labels='inferred',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    interpolation='bilinear',
    batch_size=batch_size,
    cros_to_aspect_ratio =False,
    follow_links=False,
    shuffle=False
)

Found 41961 validated image filenames belonging to 14 classes.
Found 10490 validated image filenames belonging to 14 classes.
Found 17484 validated image filenames belonging to 14 classes.


In [6]:
class_names = train_generator.class_indices
class_num =len(class_names)
print(class_names, class_num)

{'balkon': 0, 'banyo': 1, 'bar': 2, 'dış': 3, 'havuz': 4, 'mutfak': 5, 'oda': 6, 'resepsiyon': 7, 'restoran': 8, 'salon': 9, 'sauna': 10, 'spa': 11, 'spor': 12, 'veranda': 13} 14


In [7]:
# Using InceptionV3 as the base model
base_model=InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
)
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [8]:
# Creating our layers on top of base model
inputs=Input(shape=(256,256,3))
x=base_model(inputs)
x=GlobalAveragePooling2D()(x)
outputs=Dense(14,activation="softmax")(x)

model=Model(inputs,outputs)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 14)                28686     
                                                                 
Total params: 21,831,470
Trainable params: 21,797,038
Non-trainable params: 34,432
_________________________________________________________________


In [10]:
# Adam Optimizer used for training
optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)

In [11]:
# Compiling the model
model.compile(optimizer=optimizer,
                loss=CategoricalCrossentropy(),
                metrics=[CategoricalAccuracy(),
                TopKCategoricalAccuracy(k=3)]
)

In [12]:
# Creating checkpoints
model_checkpoint=ModelCheckpoint(
    filepath="Ocak",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch"
)

In [13]:
epochs=100

In [14]:
from keras.callbacks import TensorBoard, ModelCheckpoint

tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False)


In [15]:
# Fitting the model to the data
from keras.callbacks import TensorBoard
history = model.fit(x=train_generator,
            epochs=epochs,
            steps_per_epoch=72520//128,
            verbose=1,
            callbacks=[model_checkpoint,tensorboard_callback],
            validation_data=val_generator,
            validation_steps=1475//64,
)



Epoch 1/100
566/566 [==============================] - ETA: 0s - loss: 2.6875 - categorical_accuracy: 0.1241 - top_k_categorical_accuracy: 0.3030
Epoch 1: val_loss improved from inf to 186872.34375, saving model to Ocak


INFO:tensorflow:Assets written to: Ocak\assets


INFO:tensorflow:Assets written to: Ocak\assets


566/566 [==============================] - 119s 191ms/step - loss: 2.6875 - categorical_accuracy: 0.1241 - top_k_categorical_accuracy: 0.3030 - val_loss: 186872.3438 - val_categorical_accuracy: 0.0870 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/100
566/566 [==============================] - ETA: 0s - loss: 2.6490 - categorical_accuracy: 0.0976 - top_k_categorical_accuracy: 0.2800
Epoch 2: val_loss improved from 186872.34375 to 2.55434, saving model to Ocak


INFO:tensorflow:Assets written to: Ocak\assets


...

INFO:tensorflow:Assets written to: Ocak\assets


566/566 [==============================] - 78s 138ms/step - loss: 1.9084 - categorical_accuracy: 0.3587 - top_k_categorical_accuracy: 0.6343 - val_loss: 1.7185 - val_categorical_accuracy: 0.4457 - val_top_k_categorical_accuracy: 0.7174
Epoch 74/100
566/566 [==============================] - ETA: 0s - loss: 1.9435 - categorical_accuracy: 0.3569 - top_k_categorical_accuracy: 0.6453
Epoch 74: val_loss did not improve from 1.71850
566/566 [==============================] - 46s 80ms/step - loss: 1.9435 - categorical_accuracy: 0.3569 - top_k_categorical_accuracy: 0.6453 - val_loss: 2.1426 - val_categorical_accuracy: 0.3913 - val_top_k_categorical_accuracy: 0.6848
Epoch 75/100
566/566 [==============================] - ETA: 0s - loss: 1.8845 - categorical_accuracy: 0.3781 - top_k_categorical_accuracy: 0.6542
Epoch 75: val_loss did not improve from 1.71850
566/566 [==============================] - 46s 80ms/step - loss: 1.8845 - categorical_accuracy: 0.3781 - top_k_categorical_accuracy: 0.6542

INFO:tensorflow:Assets written to: Ocak\assets


INFO:tensorflow:Assets written to: Ocak\assets


566/566 [==============================] - 78s 137ms/step - loss: 1.9188 - categorical_accuracy: 0.3578 - top_k_categorical_accuracy: 0.6555 - val_loss: 1.6387 - val_categorical_accuracy: 0.4348 - val_top_k_categorical_accuracy: 0.7283
Epoch 78/100
566/566 [==============================] - ETA: 0s - loss: 1.8417 - categorical_accuracy: 0.3794 - top_k_categorical_accuracy: 0.6762
Epoch 78: val_loss did not improve from 1.63872
566/566 [==============================] - 45s 80ms/step - loss: 1.8417 - categorical_accuracy: 0.3794 - top_k_categorical_accuracy: 0.6762 - val_loss: 1.8809 - val_categorical_accuracy: 0.4022 - val_top_k_categorical_accuracy: 0.6522
Epoch 79/100
566/566 [==============================] - ETA: 0s - loss: 1.9024 - categorical_accuracy: 0.3777 - top_k_categorical_accuracy: 0.6533
Epoch 79: val_loss did not improve from 1.63872
566/566 [==============================] - 46s 80ms/step - loss: 1.9024 - categorical_accuracy: 0.3777 - top_k_categorical_accuracy: 0.6533

In [16]:
model.save("Ocak/model_using_InceptipnV3.h5")


In [17]:
import matplotlib.pyplot as plt
fig1 = plt.gcf()
plt.plot(history.history['top_k_categorical_accuracy'])
plt.plot(history.history['val_top_k_categorical_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.savefig("Ocak/Val_Accury_bacth64_epoch30.png",dpi=200)
plt.show()

In [18]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.savefig("Ocak/Val_Loss_bacth64_epoch30.png",dpi=200)
plt.show()

In [19]:
results = model.evaluate(val_generator, verbose=0)
print(f"Test Accuracy: {np.round(results[1] * 100,2)}%")

#_,acc=model.evaluate(val_generator)
#print(acc*100)    

Test Accuracy: 41.33%


In [29]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = np.argmax(model.predict(val_generator), axis=1)
matrix = confusion_matrix(val_generator.labels, predictions)
report= classification_report(val_generator.labels, predictions, target_names=val_generator.class_indices, zero_division=0)


2623/2623 [==============================] - 121s 46ms/step


In [21]:
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

      balkon       0.00      0.00      0.00      1253
       banyo       0.00      0.00      0.00      1253
         bar       0.07      0.87      0.12      1283
         dış       0.00      0.00      0.00      1266
       havuz       0.00      0.00      0.00      1248
      mutfak       0.00      0.00      0.00      1278
         oda       0.06      0.04      0.05      1319
  resepsiyon       0.00      0.00      0.00      1221
    restoran       0.00      0.00      0.00      1265
       salon       0.00      0.00      0.00      1292
       sauna       0.00      0.00      0.00      1041
         spa       0.00      0.00      0.00      1209
        spor       0.00      0.00      0.00      1271
     veranda       0.00      0.00      0.00      1285

    accuracy                           0.07     17484
   macro avg       0.01      0.06      0.01     17484
weighted avg       0.01      0.07      0.01     17484

